# Chapter04. 다양한 분류 알고리즘

## 04-1. 로지스틱 회귀

In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split    # 훈련/테스트 세트 split
from sklearn.preprocessing import StandardScaler        # 표준화 전처리
from sklearn.neighbors import KNeighborsClassifier      # k-최근접 이웃 분류

In [12]:
# 데이터 불러오기
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [13]:
# 생선 종류 (고유값) 추출
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [14]:
# target data : Species / input data : 나머지 5개 열
fish_input = fish[['Weight','Length','Diagonal','Height','Width']]
fish_input.head()

,Weight,Length,Diagonal,Height,Width
0,242.0,25.4,30.0,11.5200,4.0200
1,290.0,26.3,31.2,12.4800,4.3056
2,340.0,26.5,31.1,12.3778,4.6961
3,363.0,29.0,33.5,12.7300,4.4555
4,430.0,29.0,34.0,12.4440,5.1340


In [15]:
fish_target = fish['Species']

In [16]:
# 훈련 세트 / 테스트 세트 분할 
# random_state=42   :  데이터를 무작위로 섞어주는 seed 값
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [17]:
# K-최근접 이웃 분류 모델의 경우 거리 기반 모델이므로 표준화 전처리
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### K-최근접 이웃 분류기의 확률 예측

In [21]:
# 최근접 이웃 개수(K)=3 으로 설정
kn = KNeighborsClassifier(n_neighbors=3)
# 훈련세트로 모델 훈련
kn.fit(train_scaled, train_target)
print("훈련 세트 점수 :", kn.score(train_scaled, train_target))
print("테스트 세트 점수 :",kn.score(test_scaled, test_target))

훈련 세트 점수 : 0.8907563025210085
테스트 세트 점수 : 0.85


타깃 데이터인 'Species'에 2개 이상의 클래스가 포함되어 있으므로 이를 다중분류라고 함

In [23]:
# 타깃값 확인
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [24]:
# 테스트 세트에 있는 처음 5개 샘플의 타깃값을 예측
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [ ]:
# 클래스별 확률값 반환
# 테스트 세트에 있는 처음 5개 샘플의 확률을 출력
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))  # 반올림하여 소수점 네 번째 자리까지 표기

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [31]:
# DataFrame으로 출력
df_proba = pd.DataFrame(
    np.round(proba, 4),
    columns = kn.classes_
)
df_proba.index = [f"sample_{i}" for i in range(5)]
# 예측 클래스 표기
pred = kn.predict(test_scaled[:5])
df_proba["predicted_class"] = pred
print(df_proba)

          Bream  Parkki   Perch  Pike   Roach  Smelt  Whitefish  \
sample_0    0.0     0.0  1.0000   0.0  0.0000    0.0        0.0   
sample_1    0.0     0.0  0.0000   0.0  0.0000    1.0        0.0   
sample_2    0.0     0.0  0.0000   1.0  0.0000    0.0        0.0   
sample_3    0.0     0.0  0.6667   0.0  0.3333    0.0        0.0   
sample_4    0.0     0.0  0.6667   0.0  0.3333    0.0        0.0   

         predicted_class  
sample_0           Perch  
sample_1           Smelt  
sample_2            Pike  
sample_3           Perch  
sample_4           Perch  


In [ ]:
# 모델의 계싼한 확률이 가장 가까운 이웃의 비율이 맞는지 확인
# 네 번째 샘플의 최근접 이웃의 클래스를 확인
# iloc : 주어진 값을 정수 인덱스로 사용해 행이나 열을 선택
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target.iloc[indexes[0]])

52     Roach
106    Perch
103    Perch
Name: Species, dtype: object
